In [ ]:
from IPython.display import display, Image, Audio
import cv2  # We're using OpenCV to read video
import base64
import time
import openai
import os
import requests

In [ ]:
## 使用'0.28.0'版本
openai.__version__

In [ ]:
your_api_key = '你的openai key'

In [ ]:
# 处理所有帧
# video = cv2.VideoCapture("lol2.mov")

# base64Frames = []
# while video.isOpened():
#     success, frame = video.read()
#     if not success:
#         break
#     _, buffer = cv2.imencode(".jpg", frame)
#     base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

# video.release()
# print(len(base64Frames), "frames read.")

In [ ]:
video = cv2.VideoCapture("lol2.mov")

base64Frames = []
frame_skip = 5*3  # 例如，只处理每第15帧
frame_count = 0

while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    if frame_count % frame_skip == 0:  # 使用模运算符来检查帧计数
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
    frame_count += 1

video.release()
print(len(base64Frames), "frames read.")

In [ ]:
display_handle = display(None, display_id=True)
for img in base64Frames:
    display_handle.update(Image(data=base64.b64decode(img.encode("utf-8"))))
    time.sleep(0.025)

In [ ]:
tmp_prompt = "这些英雄联盟比赛中精彩环节的视频的帧，假设你是一名超级兴奋的韩国英雄联盟解说，请你编写一段配音脚本，每当出现精彩画面时，可以使用兴奋的词汇，选手的名字用英文表达，其他使用中文。你的回复不允许包含和游戏无关的内容，应该尽量细节的描述选手英雄的操作以及游戏内细节，减少空泛的内容,其中双方选人细节为：lng队伍中上单zika选择的是灵罗娃娃，大野Tarzan选择的是皇子，中单scout选择的是沙皇，adc gala选择的是卡莎，辅助hang选择的是机器人，T1队伍中，上单宙斯选择的是剑魔，打野oner选择的是芮尔，中单Faker选择的是发条，adc gumayusi选择的是尼菈，辅助keria选择的是赛娜;开头介绍一下你是AlphaSue开发者基于OPENAI开发的英雄联盟解说员。"

In [ ]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            tmp_prompt,
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::10]),
        ],
    },
]
params = {
    "model": "gpt-4-vision-preview",
    "messages": PROMPT_MESSAGES,
    "api_key": your_api_key,
    "headers": {"Openai-Version": "2020-11-07"},
    "max_tokens": 1024*3,
}

result = openai.ChatCompletion.create(**params)
print(result.choices[0].message.content)

In [ ]:
result.choices[0].message.content

In [ ]:
response = requests.post(
    "https://api.openai.com/v1/audio/speech",
    headers={
        "Authorization": "Bearer {}".format(your_api_key),
    },
    json={
        "model": "tts-1",
        "input": result.choices[0].message.content,
        "voice": "nova",
    },
)

audio = b""
for chunk in response.iter_content(chunk_size=1024 * 1024):
    audio += chunk
Audio(audio)

In [ ]:
# ... your existing code ...

# Now, write the `audio` bytes to an MP3 file
with open('output.mp3', 'wb') as file:
    file.write(audio)

print("The MP3 file has been saved locally as 'output.mp3'.")
